# Welcome to Plus Explorer Data Input

This notebook is a utility for me to quickly compile a dataset for analysis.

If you were looking for the data explorer, you are probably looking for this notebook.

Related Blog post: [*Insert blog post address*]

In [4]:
import pandas as pd

dataframe = pd.DataFrame(columns=['act_name',
                                  'previous', 'previous_link',
                                  'previous_len', 'previous_flesch_reading_ease', 'previous_gunning_fog',
                                  'previous_ari', 'previous_sentence_count', 'previous_lexicon_count',
                                  'current', 'current_link',
                                  'current_len', 'current_flesch_kincaid_grade', 'current_gunning_fog', 'current_ari',
                                  'current_sentence_count', 'current_lexicon_count',
                                  ])

import ipywidgets as widgets
load_button = widgets.Button(description='Load Data', icon='Load')

def load_data(b):
    global dataframe
    dataframe = pd.read_csv("../data.csv.gz")

load_button.on_click(load_data)


load_button

Button(description='Load Data', icon='Load', style=ButtonStyle())

In [5]:
import ipywidgets as widgets
from ipywidgets import TwoByTwoLayout

act_name = widgets.Text(description='Current name of Act')

previous = widgets.Textarea(
    placeholder='Cut and paste previous version of provision',
    description='Pre 2020 Version'
)
current = widgets.Textarea(
    placeholder='Cut and paste current version of provision',
    description='2020 Version'
)
previous_link = widgets.Text(
    placeholder='Link to Pre 2020 Version'
)
current_link = widgets.Text(
    placeholder='Link to 2020 Version'
)

add_button = widgets.Button(description='Submit', icon='plus')

output = widgets.Output(layout={'border': '1px solid black'})
output_diff = widgets.Output(layout={'border': '1px solid black'})


def validate_output():
    if previous.value and previous_link.value and current.value and current_link.value:
        return True
    else:
        with output:
            print('All fields are required.')
        return False


def on_clicked(b):
    output.clear_output()
    output_diff.clear_output()
    if validate_output():
        from helpers import clean_text, calculate_stats
        output.append_stdout(f'Processing: {act_name.value} \n')
        previous_stat = calculate_stats(clean_text(previous.value))
        output.append_stdout(
            f'Previous: {previous_stat[0]} \n ({previous_stat[1]},{previous_stat[2]},{previous_stat[3]},{previous_stat[4]},{previous_stat[5]}) \n')
        current_stat = calculate_stats(clean_text(current.value))
        output.append_stdout(
            f'Current: {current_stat[0]} \n ({current_stat[1]}, {current_stat[2]}, {current_stat[3]}, {current_stat[4]}, {current_stat[5]} ) \n')

        dataframe.loc[len(dataframe.index)] = [
            act_name.value,
            previous_stat[0], previous_link.value,
            len(previous_stat[0]), previous_stat[1], previous_stat[2], previous_stat[3], previous_stat[4],
            previous_stat[5],
            current_stat[0], current_link.value,
            len(current_stat[0]), current_stat[1], current_stat[2], current_stat[3], current_stat[4], current_stat[5]
        ]

        from IPython.display import display_markdown
        from helpers import Redlines
        diff = Redlines(previous_stat[0], current_stat[0])
        with output_diff:
            display_markdown(diff.output_markdown, raw=True)

        output.append_stdout(f'Current number of records: {len(dataframe.index)} \n')
        previous.value = ''
        current.value = ''
        current_link.value = ''
        previous_link.value = ''


add_button.on_click(on_clicked)

widgets.VBox([
    act_name,
    TwoByTwoLayout(top_left=previous, top_right=current, bottom_left=previous_link, bottom_right=current_link,
                   width="50%"),
    add_button,
    output,
    output_diff
])

ModuleNotFoundError: No module named 'textstat'

In [6]:
import ipywidgets as widgets


def on_file_clicked(b):
    dataframe.to_csv("data.csv.gz", compression="gzip", index=False)
    with file_output:
        from IPython.display import display, FileLink
        display(FileLink("data.csv.gz"))


file_button = widgets.Button(description='Download Data', icon='file')
file_button.on_click(on_file_clicked)
file_output = widgets.Output()

widgets.VBox([
    file_button,
    file_output
])